<h1> DeepBF-alike Architecture for Compressive Beamforming </h1>

Let's first import all of the libraries...

In [1]:
# %matplotlib inline

import scipy.io as spio
import gc
import scipy
import matplotlib.pyplot as plt
import numpy as np
import h5py
import imageio
import cv2
import tensorflow
import keras
from sklearn.preprocessing import Normalizer
from keras.utils.vis_utils import plot_model
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.initializers import *
from keras.regularizers import *
from scipy.signal import hilbert
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.callbacks import *
from scipy import ndimage
from scipy import signal
;

/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

''

The next cell is used to define couple of loss functions used for some of the advanced applications. This is entirely up to the user whether to run, and is provided only so that there is no need to try an implement these functions again.

In [2]:
from keras import backend as K

alpha=K.constant(0.9)

def MSE(y_true, y_pred):
    return K.mean(K.square(y_pred-y_true))

def mixed_error(y_true, y_pred):
    return (alpha*K.abs(K.abs(K.mean(K.square(y_pred - y_true))))+(1-alpha)*K.mean(K.abs(y_pred - y_true)))

def log10(x):
    numerator = K.log(x)
    denominator = K.log(K.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def PSNR(y_true, y_pred):
    max_pixel = K.constant(1.0)
    return (10.0 * log10((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1))))

def one_over_psnr(y_true, y_pred):
    return 1/PSNR(y_true, y_pred)

def loss_psnr(y_true, y_pred):
    mse = K.mean(K.square(y_pred-y_true), axis=-1)
    psnr = PSNR(y_true, y_pred)#K.image.psnr(y_true, y_pred, max_val = 1.0)
    return 1 - (10*log10(1/mse))

def loss_smsle(y_true, y_pred):
#     ytp = K.gather(y_true,K.greater_equal(y_true,0))
#     ytn = K.gather(y_true,K.less(y_true,0))
#     ypp = K.gather(y_pred,K.greater_equal(y_pred,0))
#     ypn = K.gather(y_pred,K.less(y_pred,0))
    return 1/2*MSE(log10(K.relu(y_pred)+1),log10(K.relu(y_true)+1))+1/2*MSE(log10(K.relu(-y_pred)+1),log10(K.relu(-y_true)+1))

In [3]:
class Antirectifier(Layer):
    def __init__(self, initializer="he_normal", **kwargs):
        super(Antirectifier, self).__init__(**kwargs)
        self.initializer = keras.initializers.get(initializer)

    def build(self, input_shape):
        output_dim = input_shape[-1]
        self.kernel = self.add_weight(
            shape=(output_dim * 2, output_dim),
            initializer=self.initializer,
            name="kernel",
            trainable=True,
        )

    def call(self, inputs):
        inputs -= tf.reduce_mean(inputs, axis=-1, keepdims=True)
        pos = tf.nn.relu(inputs)
        neg = tf.nn.relu(-inputs)
        concatenated = tf.concat([pos, neg], axis=-1)
        mixed = tf.matmul(concatenated, self.kernel)
        return mixed

    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super(Antirectifier, self).get_config()
        config = {"initializer": keras.initializers.serialize(self.initializer)}
        return dict(list(base_config.items()) + list(config.items()))


Load the data. There are several lines of code that were being used as an experimentation for determining the appropriate normalization for this type of data. The next cell performs normalization in terms of dividing by the largest value.

The data split is up to the user, and for the sake of simplicity, the validation set is the same as test set.

In [6]:
a=0
b=1
size_train=180
size_test=20
rf_place = 'RF_realdata_4smpl_carotid/Comp8/RcvData'
img_place = 'IQData_realdata_4smpl_carotid/IQData'
length=1100
start_smpl=100
trainRF=np.empty([(length-start_smpl)*size_train,16,128,3], dtype='int32')
trainIMG=np.empty([(length-start_smpl)*size_train,2,128,1])
testRF=np.empty([(length-start_smpl)*size_test,16,128,3], dtype='int32')
testIMG=np.empty([(length-start_smpl)*size_test,2,128,1])

for size in range(size_train):
    #RcvData
    RcvData = spio.loadmat(rf_place+str(size+1)+'.mat', squeeze_me=True)
    RcvData = RcvData['compressed']#/np.max(RcvData['compressed'])    # MAYBE THIS HELPS? SEEMS LIKE IT DOESN'T
    trainRF_temp=np.empty([length-start_smpl,16,128,3])
    matrix_rf=np.empty([length-start_smpl,128,16])
    j=0
    for cnt in range(128):
        matrix_rf[:,cnt,:]=RcvData[start_smpl:length,:,cnt]
#     matrix_rf_trans=np.dstack((np.zeros([16,128,1]),matrix_rf.T,np.zeros([16,128,1])))
    matrix_rf_trans=matrix_rf.T
    for i in range(length-start_smpl-2):
        trainRF_temp[i]=matrix_rf_trans[:,:,i:i+3]
    trainRF[size*(length-start_smpl):(size+1)*(length-start_smpl)]=trainRF_temp
    
#trainRF is [num, channel_no, rayline, depth]
    
    #IQ data
    trainIMG_temp = spio.loadmat(img_place+str(size+1)+'.mat', squeeze_me=True)
    trainIMG_temp = trainIMG_temp['interData'][start_smpl:length-2,:]
    trainIMG_temp1 = np.empty([length-start_smpl-2,128,2])
    trainIMG_temp1[:,:,0]=trainIMG_temp.real
    trainIMG_temp1[:,:,1]=trainIMG_temp.imag
#     trainIMG_temp1= trainIMG_temp1[start_smpl:length,:,:]
#     trainIMG_temp1 = trainIMG_temp1 / np.absolute(trainIMG_temp1).max()
#     trainIMG_temp1 = (trainIMG_temp1-trainIMG_temp1.min())/(trainIMG_temp1.max()-trainIMG_temp1.min())
#     trainIMG_temp1 = a+((trainIMG_temp1-np.absolute(trainIMG_temp).min())*(b-a))/(np.absolute(trainIMG_temp).max()-np.absolute(trainIMG_temp).min())      # THIS MAKES A MESS! WHY? IT works w abs
#     trainIMG_temp1 = (trainIMG_temp1 >= 0.0032)*trainIMG_temp1 + 0.0032*(trainIMG_temp1<0.0032)
    trainIMG[size*(length-start_smpl-2):(size+1)*(length-start_smpl-2),0,:,0]=trainIMG_temp1[:,:,0]
    trainIMG[size*(length-start_smpl-2):(size+1)*(length-start_smpl-2),1,:,0]=trainIMG_temp1[:,:,1]
#     comp = (comp>=0.0032)*comp+0.0032*(comp<0.0032)
    
#TEST
for size in range(size_test):
    #RcvData
    RcvData = spio.loadmat(rf_place+str(size_train+size+1)+'.mat', squeeze_me=True)
    RcvData = RcvData['compressed']#/np.max(RcvData['compressed'])    # MAYBE THIS HELPS?
    trainRF_temp=np.empty([length-start_smpl,16,128,3])
    matrix_rf=np.empty([length-start_smpl,128,16])
    j=0
    for cnt in range(128):
        matrix_rf[:,cnt,:]=RcvData[start_smpl:length,:,cnt]
#     matrix_rf_trans=np.dstack((np.zeros([16,128,1]),matrix_rf.T,np.zeros([16,128,1])))
        matrix_rf_trans=matrix_rf.T
    for i in range(length-start_smpl-2):
        trainRF_temp[i]=matrix_rf_trans[:,:,i:i+3]
    testRF[size*(length-start_smpl):(size+1)*(length-start_smpl)]=trainRF_temp
    
    #IQ data
    trainIMG_temp = spio.loadmat(img_place+str(size_train+size+1)+'.mat', squeeze_me=True)
    trainIMG_temp = trainIMG_temp['interData'][start_smpl:length-2,:]
    trainIMG_temp1 = np.empty([length-start_smpl-2,128,2])
    trainIMG_temp1[:,:,0]=trainIMG_temp.real
    trainIMG_temp1[:,:,1]=trainIMG_temp.imag
#     trainIMG_temp1= trainIMG_temp1[start_smpl:length,:,:]
#     trainIMG_temp1 = trainIMG_temp1 / np.absolute(trainIMG_temp1).max()
#     trainIMG_temp1 = (trainIMG_temp1-trainIMG_temp1.min())/(trainIMG_temp1.max()-trainIMG_temp1.min())
#     trainIMG_temp1 = a+((trainIMG_temp1-np.absolute(trainIMG_temp).min())*(b-a))/(np.absolute(trainIMG_temp).max()-np.absolute(trainIMG_temp).min())      # THIS MAKES A MESS! WHY? IT works w abs
#     trainIMG_temp1 = (trainIMG_temp1 >= 0.0032)*trainIMG_temp1 + 0.0032*(trainIMG_temp1<0.0032)
    testIMG[size*(length-start_smpl-2):(size+1)*(length-start_smpl-2),0,:,0]=trainIMG_temp1[:,:,0]
    testIMG[size*(length-start_smpl-2):(size+1)*(length-start_smpl-2),1,:,0]=trainIMG_temp1[:,:,1]
    
validRF=testRF
validIMG=testIMG

In [7]:
img_max = np.absolute(trainIMG).max()
trainIMG=trainIMG/img_max
testIMG = testIMG/img_max
validIMG = validIMG/img_max

The next cell creates the neural network.

There are a lot of parameters in the first part of the cell, and they are explained in more details in the report itself. 

All of the layers are created using the "exec" function, which executes a string. This is used in order to iterate through layers using a for loop, and to apply all of the necessary modifiers and parameters.

In [9]:
input_shape=trainRF.shape
k=4
num_filters=16
num_filters_last=2
kernel_size=(1,3)
k_reg_l1=0#1e-4    # PLAY W THIS
k_reg_l2=0#1e-4
k_reg_l1_last=0#1e-4
k_reg_l2_last=0#1e-5
b_reg_l1=0
b_reg_l2=0#3e-1
act_reg_l1=0
act_reg_l2=0#1e-6
act_reg_l1_last=0
act_reg_l2_last=0#1e-6
layer_act='linear'
postnorm_act='relu'
bnorm=True
use_bias=False
inputs = Input(shape = input_shape[1:])

size=4*(k*2+1)

for i in range(1,size+1):
    if i==1:
        exec('conv'+str(i)+'=Conv2D('+str(num_filters)+',kernel_size='+str(kernel_size)+',activation="'+str(layer_act)+'",padding="same",kernel_initializer="glorot_normal",use_bias='+str(use_bias)+',data_format="channels_first", activity_regularizer=l1_l2(l1='+str(act_reg_l1)+',l2='+str(act_reg_l2)+'), kernel_regularizer=l1_l2(l1='+str(k_reg_l1)+',l2='+str(k_reg_l2)+'), bias_regularizer=l1_l2(l1='+str(b_reg_l1)+',l2='+str(b_reg_l2)+'))(inputs)')
    else:
        exec('conv'+str(i)+'=Conv2D('+str(num_filters)+',kernel_size='+str(kernel_size)+',activation="'+str(layer_act)+'",padding="same",kernel_initializer="glorot_normal",use_bias='+str(use_bias)+',data_format="channels_first", activity_regularizer=l1_l2(l1='+str(act_reg_l1)+',l2='+str(act_reg_l2)+'), kernel_regularizer=l1_l2(l1='+str(k_reg_l1)+',l2='+str(k_reg_l2)+'), bias_regularizer=l1_l2(l1='+str(b_reg_l1)+',l2='+str(b_reg_l2)+'))(act'+str((i-1))+')')
    if (i-1)%4==0 and (i-1)/4>k:
        concat_loc = ((size-i+1)//4-1)*4 + 1
#         exec('add'+str(i)+'=concatenate([conv'+str(concat_loc)+', conv'+str(i)+'], axis=1)')
        exec('add'+str(i)+'=add([conv'+str(concat_loc)+', conv'+str(i)+'])')
        if bnorm:
            exec('norm'+str(i)+'=BatchNormalization(axis=1)(add'+str(i)+')')
    else:
        if bnorm:
            exec('norm'+str(i)+'=BatchNormalization(axis=1)(conv'+str(i)+')')
    if bnorm:
        exec('act'+str(i)+'=Antirectifier()(norm'+str(i)+')')
    else:
        if (i-1)%4==0 and (i-1)/4>k:
            exec('act'+str(i)+'=Antirectifier()(add'+str(i)+')')
        else:
            exec('act'+str(i)+'=Antirectifier()(conv'+str(i)+')')

exec('final=Conv2D('+str(num_filters)+',kernel_size=(1,3),kernel_initializer="glorot_normal",use_bias='+str(use_bias)+',padding="same", data_format="channels_first", activation="linear", activity_regularizer=l1_l2(l1='+str(act_reg_l1_last)+',l2='+str(act_reg_l2_last)+'), kernel_regularizer=l1_l2(l1='+str(k_reg_l1_last)+',l2='+str(k_reg_l2_last)+'), bias_regularizer=l1_l2(l1='+str(b_reg_l1)+',l2='+str(b_reg_l2)+'))(act'+str(size)+')')

out = AveragePooling2D(pool_size=(num_filters//2,1), data_format='channels_last', padding='valid')(final)

model=Model(input = inputs, output = out)

model.summary()








__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16, 128, 3)   0                                            
__________________________________________________________________________________________________


/home/hero/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`


conv2d_1 (Conv2D)               (None, 16, 128, 3)   768         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 128, 3)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
antirectifier_1 (Antirectifier) (None, 16, 128, 3)   18          batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 16, 128, 3)   768         antirectifier_1[0][0]            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 16, 128, 3)   64          conv2d_2[0][0]                   
__________________________________________________________________________________________________
antirectif

Implementation of one-cycle SGD.

In [10]:
# Implement One Cycle Policy Algorithm in the Keras Callback Class

from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy
from keras import backend as K
from keras.callbacks import *

class CyclicLR(keras.callbacks.Callback):
    
    def __init__(self,base_lr, max_lr, step_size, base_m, max_m, cyclical_momentum):
 
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.base_m = base_m
        self.max_m = max_m
        self.cyclical_momentum = cyclical_momentum
        self.step_size = step_size
        
        self.clr_iterations = 0.
        self.cm_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
        
    def clr(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)          
            return self.base_lr-(self.base_lr-self.base_lr/100)*np.maximum(0,(1-x))
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0,(1-x))
    
    def cm(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1) 
            return self.max_m
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.max_m - (self.max_m-self.base_m)*np.maximum(0,(1-x))
        
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
        if self.cyclical_momentum == True:
            if self.clr_iterations == 0:
                K.set_value(self.model.optimizer.momentum, self.cm())
            else:
                K.set_value(self.model.optimizer.momentum, self.cm())
            
            
    def on_batch_begin(self, batch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        
        if self.cyclical_momentum == True:
            self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
        
        if self.cyclical_momentum == True:
            K.set_value(self.model.optimizer.momentum, self.cm())
            



Finally, training the data:

In [11]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# CLR parameters

batch_size = 336
ep = 3
max_lr = 1e-3
base_lr = 1e-7
max_m = 0.5
base_m = 0.2

cyclical_momentum = True
augment = True
cycles = 2.1

iterations = round(trainRF.shape[0]/batch_size*ep)
iterations = list(range(0,iterations+1))
step_size = len(iterations)/(cycles)


model.compile(loss=loss_smsle, optimizer=SGD(1e-4))

clr =  CyclicLR(base_lr=base_lr,
                max_lr=max_lr,
                step_size=step_size,
                max_m=max_m,
                base_m=base_m,
                cyclical_momentum=cyclical_momentum)
    
callbacks = [clr,
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',mode='min',verbose=1,save_best_only=True)]

history = model.fit(trainRF, 
                    trainIMG, 
                    epochs=ep, 
                    verbose=1, 
                    batch_size=batch_size,
                    callbacks=callbacks,
                    validation_data=(testRF, testIMG))


Train on 180000 samples, validate on 20000 samples
Epoch 1/3
180000/180000 [==============================] - 201s 1ms/step - loss: 0.0012 - val_loss: 9.5586e-04

Epoch 00001: val_loss improved from inf to 0.00096, saving model to best_model.h5
Epoch 2/3
180000/180000 [==============================] - 195s 1ms/step - loss: 0.0011 - val_loss: 8.9618e-04

Epoch 00002: val_loss improved from 0.00096 to 0.00090, saving model to best_model.h5
Epoch 3/3
180000/180000 [==============================] - 196s 1ms/step - loss: 0.0011 - val_loss: 8.8331e-04

Epoch 00003: val_loss improved from 0.00090 to 0.00088, saving model to best_model.h5


If a typical optimizers are to be used (not a one-cycle SGD as implemented above) then the following cells need to be uncommented and run.

In [12]:
# # # model = Model(input = inputs, output = out)
# ep = 5
# learning_rate = 1e-3
# decay_rate = learning_rate/(ep)
# opt = keras.optimizers.Adam(lr=learning_rate, decay = decay_rate)

# model.compile(optimizer = opt, loss = loss_psnr)

In [13]:
# # callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',mode='min',verbose=1,save_best_only=True)]

# history = model.fit(trainRF,
#                     trainIMG,
#                     epochs=ep,
#                     verbose=1,
# #                     callbacks=callbacks,
#                     batch_size=336,
#                     validation_data=(testRF, testIMG))

Optionally, a trained model can be saved.

In [14]:
# model.save('sgd_one_cycle_ksize_3_3_200ep_carotid_diffnormalization')
# model=load_model('best_model.h5', custom_objects={ 'loss_psnr': loss_psnr })

This is the verificaiton part. Apart from the loss that we can observe throughout the training, a resulting image needs to be evaluated.

In [15]:
pp=model.predict(validRF, verbose=1, batch_size=336)

20000/20000 [==============================] - 8s 383us/step


In [16]:
pp1=pp[:,:,:,0]
pp2=pp1[:,0,:]+1j*pp1[:,1,:]
pp3=np.absolute(pp2)
testIMG2=testIMG[:,:,:,0]
testIMG3=testIMG2[:,0,:]+1j*testIMG2[:,1,:]
testIMG4=np.absolute(testIMG3)
validIMG2=validIMG[:,:,:,0]
validIMG3=validIMG2[:,0,:]+1j*validIMG2[:,1,:]
validIMG4=np.absolute(validIMG3)

In [17]:
# Image Plotting
pic_no=10
start=0
end=1000
offset=100#12*10
dyn_range=-70
dyn_range_comp=-70
dr_db=10**(dyn_range/20)
dr_db_comp=10**(dyn_range_comp/20)
%pylab
comp = pp3[pic_no*length+start+offset:(pic_no)*length+end+offset]
comp = comp/np.max(comp)
# comp=1-comp
comp = (comp>=dr_db_comp)*comp+dr_db_comp*(comp<dr_db_comp)

orig = validIMG4[pic_no*length+start:(pic_no)*length+end]
orig = orig/np.max(orig)
orig = (orig>=dr_db)*orig+dr_db*(orig<dr_db)



plt.figure()#figsize=(200,10))
# axes1=fig1.add_axes([0,0,end-start, 128])
plt.imshow(orig, extent=[0,128,end,start], aspect=0.1)#, cmap=plt.cm.get_cmap('gray'))
# plt.imshow(orig)#, cmap=plt.cm.get_cmap('gray'))
plt.figure()#figsize=(200,10))
plt.imshow(comp, extent=[0,128,end,start], aspect=0.1)#, cmap=plt.cm.get_cmap('gray'))#, cmap=plt.cm.get_cmap('gray'))
# plt.show()
# plt.figure()
# plt.imshow(orig-comp,extent=[0,128,end,start], aspect=0.1)
;

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/home/hero/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['maximum', 'copy', 'average', 'dot', 'size', 'ones', 'multiply', 'concatenate', 'zeros', 'log10', 'get', 'add', 'identity', 'normal', 'minimum', 'subtract', 'uniform']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


''